**Лабораторный практикум по курсу «Распознавание диктора», Университет ИТМО, 2021**		

**Лабораторная работа №1. Информативные признаки речевых сигналов: извлечение признаков**

**Цель работы:** изучение процедуры построения информативных акустических признаков для речевых сигналов.

**Содержание лабораторной работы**

1. Выполнить процедуру высокочастотной фильтрации (преэмфазис) речевого сигнала.

2. Вычислить акустические признаки разных видов.

3. Выполнить локальные центрирование и масштабирование акустических признаков.

4. Построить распределение первых 3 компонент полученных акустических признаков для мужских и женских голосов.

In [1]:
# Import of modules
import os
import sys

sys.path.append(os.path.realpath('..'))

from common import download_dataset, extract_dataset

**1. Подготовка данных для обучения и тестирования детектора речевой активности**

В ходе выполнения лабораторной работы необходимы данные для осуществления процедуры вычисления акустических признаков. Возьмём в качестве этих данных несколько звукозаписей голосов людей мужского и женского пола, сохраненных в формат *wav*, выбранных из корпуса [VoxCeleb1 test set](https://thor.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox1_test_wav.zip). Данный корпус содержит 4,874 звукозаписи (частота дискретизации равна 16кГц) 40 дикторов мужского и женского пола, разговаривающих на английском языке.

В рамках настоящего пункта требуется выполнить загрузку и распаковку звуковых wav-файлов из корпуса VoxCeleb1 test set.

![Рисунок 1](https://analyticsindiamag.com/wp-content/uploads/2020/12/image.png "VoxCeleb. Крупномасштабная аудиовизуальная база данных человеческой речи.")

In [2]:
# Download VoxCeleb1 (test set)
with open('../data/lists/datasets.txt', 'r') as f:
    lines = f.readlines()

download_dataset(lines, user='voxceleb1902', password='nx0bl2v2', save_path='../data')

Checksum successful vox1_test_wav.zip.


In [3]:
# Extract VoxCeleb1 test set
extract_dataset(save_path='../data/voxceleb1_test', fname='../data/vox1_test_wav.zip')

Extracting of ../data/vox1_test_wav.zip is successful.


**2. Выполнить процедуру высокочастотной фильтрации (преэмфазис) речевого сигнала**

В рамках настоящего пункта предлагается изучить процедуру предобработки речевых сигналов, получившую название преэмфазис. *Преэмфазис (pre-emphasis)* – применение фильтра верхних частот к сигналу до процедуры извлечения признаков для того, чтобы компенсировать тот факт, что обычно вокализованная речь содержит на низких частотах намного больше энергии, чем невокализованная речь на высоких. Обозначим через $x(n)$ обрабатываемый сигнал, а через $y(n)$ результат обработки, тогда процедура преэмфазиса может быть описана с помощью следующего выражения:

$$y(n) = x(n) + \alpha x(n-1).$$

Выражение выше представляет собой *линейное разностное уравнение с постоянными коэффициентами* и позволяет описать процедуру работы *нерекурсивного фильтра первого порядка с конечной импульсной характеристикой*. Параметр $\alpha$ является единственным параметром рассматриваемого фильтра. В случае, когда параметр $\alpha$ является отрицательным, фильтр рассматриваемый фильтр обладает свойствами *фильтра верхних частот*. Для выполнения процедуры преэмфазиса параметр $\alpha$ обычно выбирается равным величине $-0.97$.

В рамках настоящего пункта требуется:

1. Загрузить анализируемый речевой сигнал.

2. Выполнить процедуру преэмфазиса по отношению к загруженному речевому сигналу.

3. Сравнить спектры речевых сигналов до и после процедуры преэмфазиса.

**3. Вычислить акустические признаки разных видов**

Работа современных систем голосовой биометрии, как правило, является основанной на использовании акустических признаков. *Акустические признаки* являются формой представления речевого сигнала в частотно-временной области. Возможным примером акустических признаков является *спектрограмма* сигнала, которую можно вычислить с использованием *оконного преобразования Фурье*. Однако, использование спектрограммы на практике не является выгодным, в частности, из-за её большого разрешения по частоте. Поэтому возможным вариантом для построения акустических признаков могут являться: *логарифмов энергий на выходе мел-банка фильтров*, *мел-частотные кепстральные коэффициенты размерности* и т.п.

В рамках настоящего пункта предлагается вычислить логарифмы энергий на выходе мел-банка фильтров размерности 40 и мел-частотные кепстральные коэффициенты размерности 23. Для выполнения задания выше можно воспользоваться следующим алгоритмом:

1. Представить обрабатываемый речевой сигнал в виде набора фреймов с использованием *окна Хэмминга* (размер окна можно выбрать равным 25мс, а шаг окна – 10мс).

2. Вычислить одномерный спектр Фурье по отношению к каждому из фреймов.

3. Рассчитать мел-банк фильтров.

4. Перемножить квадрат *амплитудно-частотной характеристики (АЧХ)* каждого фильтра со спектром мощности каждого из фреймов ⁡и просуммировать коэффициенты получившихся спектров, рассчитав энергии внутри соответствующих полос банка фильтров.

5. Вычислить логарифм от значений энергий на предыдущем шаге. *На этом шаги формируется первый тип акустических признаков.*

6. Вычислить дискретное косинусное преобразование от логарифмов коэффициентов. *На этом этапе формируются мел-частотные кепстральные коэффициенты.*

Подробное описание процедуры вычисления акустических признаков можно найти по следующей [ссылке](http://practicalcryptography.com/miscellaneous/).

**4. Выполнить локальные центрирование и масштабирование акустических признаков**

Канал связи может вносит некоторое смещение в захваченный сигнал (микрофон может не иметь равномерную АЧХ, изменения в усилении
сигнал приводят к вычислению различных акустических признаков даже для одного и того же куска речи). Канальный эффект можно промоделировать с использованием линейного инвариантного к сдвигу фильтра (ЛИС-фильтра) и учесть при корректировке значений акустических признаков. Данная процедура получила название *процедуры нормализации*.

Идея выполнения процедуры нормализации состоит в вычислении среднего вектора наблюдаемых акустических признаков и центрирования всех векторов акустических признаков произнесения на это среднее. Процедура нормализации может быть выполнена несколькими различными способами:

1. Локально по произнесению (вычисляем среднее в некоторой окрестности (обычно 300 фреймов) каждого фрейма стека признаков и нормализуем на них этот фрейм).

2. Глобально по произнесению (вычисляем среднее один раз по всей сессии и нормализуем на них все фреймы сессии).

3. глобально по базе данных (вычисляем среднее общее для всех сессий в тренировочной базе данных и нормализуем на них все фреймы всех сессий).

Иногда процедура нормализации сопровождается процедурой *масштабирования акустических признаков*. Процедура масштабирования признаков может быть выполнена теми же способами, что и процедура нормализации. Отличие процедуры масштабирования признаков от процедуры нормализации состоит в том, что при её выполнении необходимо вычислить не средний вектор акустических признаков, а вектор среднеквадратического отклонения, на который необходимо поэлементно разделить каждый нормализованный вектор акустических признаков некоторого произнесения.

Предполагая, что набор акустических признаков до выполнения процедур нормализации и масштабирования был задан функцией $|X(k, m)|$, а после выполнения данных процедур – $|X_{norm}(k, m)|$, запишем следующее выражение:

$$|X_{norm}(k, m)| = \frac{|X_{norm}(k, m)| - m_X}{\sigma_X},$$

где $m_X$ – это средний вектор акустических признаков, $\sigma_X$ – это вектор среднеквадратического отклонения акустических признаков, $k$ и $m$ имеют суть частоты и времени (k – номер спектральной составляющей, а m – это номер фрейма).

В рамках настоящего пункта требуется выполнить процедуры нормализации и масштабирования по отношению к логарифмам энергий на выходе банка фильтров и мел-частотным кепстральным коэффициентам, вычисленным для некоторой звукозаписи.

**5. Построить распределение первых 3 компонент полученных акустических признаков для мужских и женских голосов**

Для того, чтобы грубо проверить правильность расчёта акустических признаков выше, построим гистограммы распределения первых 3 компонент логарифмов энергий на выходе банка фильтров и мел-частотных кепстральных коэффициентов по некоторой базе данных. Рассмотрим в качестве этой базы звукозаписи мужских и женских голосов дикторов, список которых представлен в **./metadata/meta.txt**. В указанном файле перечислен список из 20 звукозаписей (10 для дикторов женского пола и 10 для дикторов мужского пола) из базы VoxCeleb1 test set с указанием пути до них, идентификационного номера диктора и пола диктора.

Используя звукозаписи, список которых перечислен в **./metadata/meta.txt**, выполнить построение гистограмм распределения первых 3 компонент логарифмов энергий на выходе банка фильтров и мел-частотных кепстральных коэффициентов отдельно для базы мужских и женских голосов.

**6. Контрольные вопросы**

1. Какие способы представления сигналов существуют?

2. Что такое спектр Фурье (амплитудный и фазовый)?

3. Что такое оконное преобразование Фурье?

4. Что такое спектрограмма?

5. Как выполнить процедуру преэмфазиса?

6. Описать процедуру вычисления акустических признаков.

7. Для каких целей выполняются процедуры нормализации и масштабирования акустических признаков?

**7. Список литературы**

1. Hayes M.H. Schaum’s outlines of digital signal processing. McGraw-Hill, 2011 ([ссылка](http://index-of.co.uk/DSP-Collection/Hayes,%20M.H.,%20Schaum's%20Outline%20of%20Digital%20Signal%20Processing,%20McGraw-Hill,%201999.pdf)).

2. Beigi H. Fundamentals of speaker recogniton. Springer, 2011 ([ссылка](https://www.springer.com/gp/book/9780387775913)).

### Лабораторная работа №1. Информативные признаки речевых сигналов: извлечение признаков
**Описание**:
В рамках данной лабораторной работы вам предстоит познакомиться с процедурами предобработки речевых сигналов и извлечения информативных признаков. В данной работе вам предлагается научиться извлекать

- Mel Filter Banks - кратковременные энергии мелчастотных полос
- Mel Frequency Cepstral Coeffitients - Мел частотные кепстральные коэффициенты

Признаки должны совпадать с соответствующими признаками, которые извлекаются в **torchaudio** (https://pytorch.org/audio/stable/transforms.html)

**Данные**:
В качестве данных для проведения исследования предлагается взять базу VoxCeleb1: http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1.html.

**Задания**:
1. Сделать preemphasis (фильтрация верхних частот). Проанализировать амлитудно-частотные спектры исходного и обработанного речевых участков сигналов
2. Вычислить акустические признаки разных видов: FBs40 и MFCCs23.
3. Выполнить локальное центрирование и масштабирование признаков (нормализация MVN).
4. Построить распределение первых 3 компонент полученных признаков по всей базе отдельно по гендерам

### 1. Preemphasis preprocessing

In [2]:
from audiofeatures import Audiofeatures
processed_waveform = Audiofeatures.preemphasis(waveform)

NameError: name 'waveform' is not defined

### 2. Mel Filter Banks computing

In [3]:
MFB40_FeaturesExctractor = Audiofeatures(feats_type='MFB40')
MFB40 = MFB40_FeaturesExctractor.extract('wav_path')

NotImplementedError: 

### 3. Mel Frequency Cepstral Coeffitiens computing

In [ ]:
MFCC_FeaturesExctractor = Audiofeatures(feats_type='MFCC')
MFCC = MFCC_FeaturesExctractor.extract('wav_path')